In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from ridgereliability.plots import posterior_reliability_diagram, confidence_reliability_diagram
from ridgereliability import metrics

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(30, kernel_size=(3, 3), padding='valid', activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(tf.keras.layers.Conv2D(13, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(120, activation='relu'))
model.add(tf.keras.layers.Dense(86, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype(np.float32) / 255
x_test = x_test.astype(np.float32) / 255

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-3,
    patience=50,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
)
callbacks = [early_stopping]

model.fit(x_train, y_train,
          batch_size=32, epochs=10,
         validation_data=(x_test, y_test),
         callbacks=callbacks)

In [ ]:
model.save('models/cifar10_lenet.h5')

In [3]:
model = tf.keras.models.load_model('models/cifar10_lenet.h5')
z_test = model.predict(x_test, batch_size=128, verbose=1)

10000/10000 [==============================] - 1s 87us/sample


In [4]:
y_probs = z_test.max(axis=1)
y_preds = z_test.argmax(axis=1)
y_true = y_test.argmax(axis=1)

In [ ]:
acc = (y_preds == y_true).mean()
print(f'Accuracy: {acc}')

ece = metrics.ece(y_probs, y_preds, y_true)
print(f'ECE     : {ece}')

peace = metrics.peace(y_probs, y_preds, y_true)
print(f'PEACE   : {peace}')

In [ ]:
ax = plt.gca()
plt.title('LeNet-5 on CIFAR-10')
posterior_reliability_diagram(y_probs, y_preds, y_true, ax, bins=10)
plt.savefig('plots/lenet.pdf')
plt.show()